<a href="https://colab.research.google.com/github/rahul-premanand/Hello-World/blob/main/preprocess_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```

# **Data Preprocessing**

# **Load Data**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')
ozone_df = pd.read_csv('/content/drive/My Drive/dataset/hourly_ozone.csv')

# Load filter pack weekly data
filter_pack_df = pd.read_csv('/content/drive/My Drive/dataset/filterpack.csv')
print(ozone_df.columns)
print(filter_pack_df.columns)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Index(['Site ID', 'Ozone', 'Units', 'QA Code', 'Update_Date', 'Ozone F',
       'Selected Date_Time'],
      dtype='object')
Index(['SITE_ID', 'Week', 'Year', 'DATEOFF', 'Ca', 'Cl', 'HNO3', 'HNO3 PPB',
       'K', 'Mg', 'Na', 'NH4', 'NO3', 'SO2', 'SO2 PPB', 'SO4', 'TNO3',
       'DATEON'],
      dtype='object')


<ipython-input-6-22092254432a>:10: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  ozone_df = pd.read_csv('/content/drive/My Drive/dataset/hourly_ozone.csv')


In [ ]:
ozone_df = ozone_df.apply(lambda x: x.fillna(x.mean()) if x.dtype.kind in 'biufc' else x)
filter_pack_df = filter_pack_df.apply(lambda x: x.fillna(x.mean()) if x.dtype.kind in 'biufc' else x)
print(ozone_df.dtypes)
print(filter_pack_df.columns)



Site ID               object
Ozone                 object
Units                 object
QA Code               object
Update_Date           object
Ozone F               object
Selected Date_Time    object
dtype: object
Index(['SITE_ID', 'Week', 'Year', 'DATEOFF', 'Ca', 'Cl', 'HNO3', 'HNO3 PPB',
       'K', 'Mg', 'Na', 'NH4', 'NO3', 'SO2', 'SO2 PPB', 'SO4', 'TNO3',
       'DATEON'],
      dtype='object')


In [ ]:
# Convert 'Update_Date' and 'DATEON' to datetime
ozone_df['Update_Date'] = pd.to_datetime(ozone_df['Update_Date'], errors='coerce')
filter_pack_df['DATEON'] = pd.to_datetime(filter_pack_df['DATEON'], errors='coerce')

# Convert 'Ozone' column to numeric type
ozone_df['Ozone'] = pd.to_numeric(ozone_df['Ozone'], errors='coerce')

# Identify numeric columns
numeric_cols_ozone = ozone_df.select_dtypes(include=['number']).columns
print("Numeric columns identified:", numeric_cols_ozone)

# Fill missing values for numeric columns only
ozone_df[numeric_cols_ozone] = ozone_df[numeric_cols_ozone].fillna(ozone_df[numeric_cols_ozone].mean())

# Set 'Update_Date' as index for resampling
ozone_df.set_index('Update_Date', inplace=True)

# Resample to weekly frequency and calculate mean
ozone_weekly_df = ozone_df[numeric_cols_ozone].resample('W-Mon').mean().reset_index()

# Print the first 20 rows of the resampled DataFrame
print(ozone_weekly_df.head(20))


Numeric columns identified: Index(['Ozone'], dtype='object')
   Update_Date      Ozone
0   2021-01-04  25.550738
1   2021-01-11  27.418262
2   2021-01-18  27.353107
3   2021-01-25  29.814612
4   2021-02-01  31.234075
5   2021-02-08  33.811018
6   2021-02-15  30.838379
7   2021-02-22  35.733216
8   2021-03-01  35.302611
9   2021-03-08  37.290608
10  2021-03-15  38.796711
11  2021-03-22  36.090957
12  2021-03-29  34.514248
13  2021-04-05  39.050739
14  2021-04-12  37.647201
15  2021-04-19  34.226054
16  2021-04-26  38.491524
17  2021-05-03  36.495881
18  2021-05-10  33.794275
19  2021-05-17  36.335484


In [ ]:


filter_pack_df.rename(columns={'DATEON': 'Update_Date'}, inplace=True)

ozone_weekly_df.drop_duplicates(subset=['Update_Date'], inplace=True)
filter_pack_df.drop_duplicates(subset=['Update_Date'], inplace=True)

# Set 'Update_Date' as the index for both dataframes
ozone_weekly_df.set_index('Update_Date', inplace=True)
filter_pack_df.set_index('Update_Date', inplace=True)

# Concatenate the dataframes along the columns using an outer join to keep all dates
concatenated_df = pd.concat([ozone_weekly_df, filter_pack_df], axis=1, join='outer')

# Print the concatenated dataframe
display(concatenated_df.head(20))
concatenated_df.to_csv('fresh_data.csv', index=False)


,Ozone,SITE_ID,Week,Year,DATEOFF,Ca,Cl,HNO3,HNO3 PPB,K,Mg,Na,NH4,NO3,SO2,SO2 PPB,SO4,TNO3
Update_Date,,,,,,,,,,,,,,,,,,
2021-01-04,25.550738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-05,NaN,CND125,2.0,2021.0,1/12/2021 8:00:00 AM,0.036,0.035,0.379,0.147,0.052,0.012,0.036,0.832,1.166,0.345,0.132,1.318,1.539
2021-01-11,27.418262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-12,NaN,CND125,3.0,2021.0,1/19/2021 8:00:00 AM,0.053,0.036,0.549,0.213,0.072,0.01,0.028,0.703,1.4,0.275,0.105,0.855,1.941
2021-01-18,27.353107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-19,NaN,CND125,4.0,2021.0,1/26/2021 8:00:00 AM,0.088,0.034,0.452,0.175,0.061,0.016,0.049,0.227,0.427,0.411,0.157,0.445,0.872
2021-01-25,29.814612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-26,NaN,CND125,5.0,2021.0,2/02/2021 8:00:00 AM,0.028,0.035,0.446,0.173,0.046,0.009,0.035,0.367,0.479,0.49,0.187,0.659,0.918
2021-02-01,31.234075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

# Check for numeric columns
numeric_columns = concatenated_df.select_dtypes(include=['float64', 'int64']).columns


# Check if there are any numeric columns to scale
if len(numeric_columns) == 0:
    raise ValueError("No numeric columns found in the DataFrame.")

# Check if there are any rows in the DataFrame
if concatenated_df.shape[0] == 0:
    raise ValueError("The DataFrame is empty. No data to scale.")

# Standardize the numeric features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(concatenated_df[numeric_columns])

# Create a DataFrame with the scaled features
scaled_df = pd.DataFrame(scaled_features, columns=numeric_columns, index=concatenated_df.index)

# Print the scaled dataframe to verify
print(scaled_df.head())

                Ozone      Week  Year
Update_Date                          
2021-01-04  -1.633875       NaN   NaN
2021-01-05        NaN -1.699056   0.0
2021-01-11  -1.119280       NaN   NaN
2021-01-12        NaN -1.632427   0.0
2021-01-18  -1.137234       NaN   NaN


In [ ]:
# Print the original Ozone values for comparison
print("Original Ozone Values:")
print(concatenated_df[['Ozone']].head(10))

# Print the scaled Ozone values
print("Scaled Ozone Values:")
scaled_df['Ozone'].fillna(value=0, inplace=True)  # Example: fill with 0
print(scaled_df[['Ozone']].head(10))

# Print statistics of the scaled Ozone data
print("Scaled Ozone Statistics:")
print(scaled_df[['Ozone']].describe())


Original Ozone Values:
                 Ozone
Update_Date           
2021-01-04   25.550738
2021-01-05         NaN
2021-01-11   27.418262
2021-01-12         NaN
2021-01-18   27.353107
2021-01-19         NaN
2021-01-25   29.814612
2021-01-26         NaN
2021-02-01   31.234075
2021-02-02         NaN
Scaled Ozone Values:
                Ozone
Update_Date          
2021-01-04  -1.633875
2021-01-05   0.000000
2021-01-11  -1.119280
2021-01-12   0.000000
2021-01-18  -1.137234
2021-01-19   0.000000
2021-01-25  -0.458967
2021-01-26   0.000000
2021-02-01  -0.067836
2021-02-02   0.000000
Scaled Ozone Statistics:
              Ozone
count  1.780000e+02
mean  -2.295293e-16
std    6.422066e-01
min   -1.881787e+00
25%    0.000000e+00
50%    0.000000e+00
75%    0.000000e+00
max    2.124491e+00


In [12]:
if 'Update_Date' in concatenated_df.index.names:
    concatenated_df.reset_index(inplace=True)

# Replace 0 with NaN in the DataFrame
concatenated_df.replace(0, np.nan, inplace=True)

# Define the feature matrix X and target vector y
X = concatenated_df.drop(columns=['Ozone'])
y = concatenated_df['Ozone']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Save preprocessed data
concatenated_df.to_csv('final_preprocessed_data.csv', index=False)

# Print to verify
display(concatenated_df.head())
print("Preprocessing completed successfully.")

,Update_Date,Ozone,SITE_ID,Week,Year,DATEOFF,Ca,Cl,HNO3,HNO3 PPB,K,Mg,Na,NH4,NO3,SO2,SO2 PPB,SO4,TNO3
0,2021-01-04,25.550738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-05,NaN,CND125,2.0,2021.0,1/12/2021 8:00:00 AM,0.036,0.035,0.379,0.147,0.052,0.012,0.036,0.832,1.166,0.345,0.132,1.318,1.539
2,2021-01-11,27.418262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-12,NaN,CND125,3.0,2021.0,1/19/2021 8:00:00 AM,0.053,0.036,0.549,0.213,0.072,0.01,0.028,0.703,1.4,0.275,0.105,0.855,1.941
4,2021-01-18,27.353107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Preprocessing completed successfully.
